In [126]:
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch
import torchvision
import torchvision.transforms as transforms

from torch.utils.data.dataset import Dataset
from torchvision import transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

print(device)

cpu


In [127]:
data = np.loadtxt('dataWhole.txt', dtype=str, delimiter = '\n')


In [128]:
blackTurnBoard = np.ones(shape=(15, 15), dtype=np.int8)
whiteTurnBoard = -np.ones(shape=(15, 15), dtype=np.int8)

In [129]:
def toTurn(turn):
    letter = ord(turn[0]) - ord('a')
    num = int(turn[1:]) - 1
    return letter, num

In [130]:
def toBoards(game, color, turn):
    if color == "black":
        playerBoard = blackTurnBoard
    else:
        playerBoard = whiteTurnBoard
        
    curPlayer = 1
    whiteBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    blackBoard = np.zeros(shape=(15, 15), dtype=np.int8)
    
    for i in range(1, turn + 1):
        turnC = toTurn(game[i])
        if curPlayer == 1:
            blackBoard[turnC] = 1
        else:
            whiteBoard[turnC] = -1
        curPlayer *= -1
    curBoard = np.zeros(shape=(3, 15, 15))
    curBoard[0,:] = playerBoard
    curBoard[1,:] = blackBoard
    curBoard[2,:] = whiteBoard
    label = toTurn(game[turn + 1])
    return curBoard, label

In [131]:
def printBoard(board):
    for i in range(14, -1, -1):
        for j in range(15):
            if board[1, i, j] != 0:
                print(" 1",  end='')
            elif board[2, i, j] != 0:
                print("-1", end='')
            else:
                print(" 0", end='')
        print()
    print()

In [156]:
def takeRandom():
    game = []
    while len(game) <= 4:
        gameNum = np.random.randint(data.shape[0])
        game = data[gameNum].split()
    color = game[0]
    turn = np.random.randint((len(game) - 2) // 2)
    if color[0] == 'b':
        board, label = toBoards(game, color, turn * 2)
    else:
        board, label = toBoards(game, color, turn * 2 + 1)

    return board, label[0] * 15 + label[1]

In [157]:
def makeBatch(size):
    batch = np.zeros(shape=(size, 3, 15, 15))
    labels = np.zeros(shape=size, dtype=np.long)
    for i in range(size):
        batch[i,:,:,:], labels[i] = takeRandom()
    return torch.from_numpy(batch), torch.from_numpy(labels)

In [158]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.batchnorm = nn.BatchNorm2d(3, affine=False)
        self.pad2 = nn.ConstantPad2d(2, 0)
        self.pad1 = nn.ConstantPad2d(1, 0)

        self.conv1 = nn.Conv2d(3, 6, 3)
        self.conv2 = nn.Conv2d(6, 4, 5)
        self.conv3 = nn.Conv2d(4, 2, 5)
        self.conv4 = nn.Conv2d(2, 1, 3)

    def forward(self, x):
        x = self.batchnorm(x.float())
        x = self.pad1(F.relu(self.conv1(x)))
        x = self.pad2(F.relu(self.conv2(x)))
        x = self.pad2(F.relu(self.conv3(x)))
        x = self.pad1(F.relu(self.conv4(x)))

        x = x.view(-1, 225)
 
        return x

In [159]:
net = Net()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
batchNum = 100000
epochNum = 10
batchSize = 32
optimizer = optim.Adam(net.parameters(), lr=0.001)
#optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [ ]:
for epoch in range(epochNum):  # loop over the dataset multiple times
    running_loss = 0.0
    for i in range(batchNum):
        # get the inputs
        inputs, labels = makeBatch(batchSize)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 4.568
[1,   400] loss: 4.154
[1,   600] loss: 4.014
[1,   800] loss: 3.961
[1,  1000] loss: 3.857
[1,  1200] loss: 3.792
[1,  1400] loss: 3.703
[1,  1600] loss: 3.686
[1,  1800] loss: 3.662
[1,  2000] loss: 3.653
[1,  2200] loss: 3.627
[1,  2400] loss: 3.584
[1,  2600] loss: 3.567
[1,  2800] loss: 3.566
[1,  3000] loss: 3.529
[1,  3200] loss: 3.528
[1,  3400] loss: 3.546
[1,  3600] loss: 3.484
[1,  3800] loss: 3.507
[1,  4000] loss: 3.498
[1,  4200] loss: 3.466
[1,  4400] loss: 3.466
[1,  4600] loss: 3.461


## Game

In [126]:
def toTripleBoard(board, side):
    curBoard = np.zeros(shape=(3, 15, 15), dtype=np.float)
    blackBoard = np.zeros(shape=(15, 15), dtype=np.float)
    whiteBoard = np.zeros(shape=(15, 15), dtype=np.float)

    if side == 1:
        blackTurnBoard = np.ones(shape=(15, 15), dtype=np.float)
    else:
        blackTurnBoard = -np.ones(shape=(15, 15), dtype=np.float)
    curBoard[0,:] = blackTurnBoard
    for i in range(15):
        for j in range(15):
            if board[i, j] == 1:
                blackBoard[i, j] = 1
            if board[i, j] == -1:
                whiteBoard[i, j] = -1
    curBoard[1,:] = blackBoard
    curBoard[2,:] = whiteBoard
    return curBoard


In [176]:
def printBoard(curBoard):
    print("-------------------------------------------------------------")
    cprint("    0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 ", "blue")
    for i in range(15):
        if i < 10:
            cprint("", i, "blue", end=" ")
        for j in range(15):
            if curBoard[i, j] == -1:
                print("|", end="")
                cprint(str(-1), 'red', "on_green", attrs=['blink'], end=" ")
            else:
                print("|", end = " ")
                if curBoard[i, j] == 1:
                    cprint(int(curBoard[i, j]), 'yellow', "on_blue", attrs=['blink'], end=" ")
                else:
                    print(int(curBoard[i, j]), end=" ")
        print("|")
        print("-------------------------------------------------------------")
    print()
    return


 ## (we are white)

In [ ]:
curBoard = np.zeros(shape=(15, 15), dtype=np.float)

while True:
    board = toTripleBoard(curBoard, 1)
    with torch.no_grad():
        outputs = net(torch.unsqueeze(torch.from_numpy(board), 0))
        _, netTurn = torch.max(outputs, 1)

    curBoard[netTurn // 15, netTurn % 15] = 1
    printBoard(curBoard)
    
    x, y = list(map(int, input().split()))
    curBoard[x, y] = -1
    #printBoard(curBoard)


-------------------------------------------------------------
    0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 
0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
5 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
6 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
------------------------------------------------------

10 13
-------------------------------------------------------------
    0   1   2   3   4   5   6   7   8   9   10  11  12  13  14 
0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
2 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
3 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
5 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
-------------------------------------------------------------
6 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 |
------------------------------------------------

## (we are black)

In [ ]:
curBoard = np.zeros(shape=(15, 15), dtype=np.float)

while True:
    x, y = list(map(int, input().split()))
    curBoard[x, y] = 1
    #printBoard(curBoard)
    
    board = toTripleBoard(curBoard, 1)
    with torch.no_grad():
        outputs = net(torch.unsqueeze(torch.from_numpy(board), 0))
        _, netTurn = torch.max(outputs, 1)

    curBoard[netTurn // 15, netTurn % 15] = -1
    #printBoard(curBoard)


In [152]:

text = colored('Hello, World!', 'red', attrs=['reverse', 'blink'])
print(text)
cprint('Hello, World!', 'green', 'on_yellow', attrs=['blink'])

Hello, World!
Hello, World!
